<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/Glove_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialization

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
from scipy import spatial
from sklearn.utils import shuffle 
from sklearn.model_selection import KFold

In [2]:
from google.colab import  drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Import

In [3]:
df_train = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/Data/Train/clean_all_train.csv')

df_train.head()

,Unnamed: 0,Tweets,Labels,Category
0,0,How the fuck Who the heck moved my fridge ...,0.938,anger
1,1,So my Indian Uber driver just called someone t...,0.896,anger
2,2,UK I asked for my parcel to be delivered to a...,0.896,anger
3,3,so ef whichever butt wipe pulled the fire alar...,0.896,anger
4,4,do not join they put the phone down on you ta...,0.896,anger


In [4]:
def find_length(tweet):

  return len(tweet.split())

In [5]:
def remove_numbers(tweet):
  tweet = re.sub('[0-9]+','',tweet)
  return tweet

In [6]:
hash={"anger":[1,0,0,0],"sadness":[0,1,0,0],"joy":[0,0,1,0],"fear":[0,0,0,1]}
def number_category(category):
  return hash[category]

In [7]:
df_train['Encoding'] = df_train['Category'].apply(lambda category : number_category(category))
df_train['Length'] = df_train['Tweets'].apply(lambda tweet : find_length(tweet))
df_train['Tweets'] = df_train['Tweets'].apply(lambda tweet : remove_numbers(tweet))

maxLengthOfTweet = df_train['Length'].max()


#shuffal data

In [8]:
def shuffle_efficiently(df_train):
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train.reset_index(drop=True, inplace=True)

  df_train.head(1000)

  return df_train

In [9]:
df_train = shuffle_efficiently(df_train)

df_train.head()

,Unnamed: 0,Tweets,Labels,Category,Encoding,Length
0,3221,et But the end of all things is at hanTong...,0.479,sadness,"[0, 1, 0, 0]",28
1,2216,The neighbor dancing in the Clayton Homes comm...,0.640,joy,"[0, 0, 1, 0]",11
2,189,All hell is breaking loose in Charlotte Char...,0.625,anger,"[1, 0, 0, 0]",10
3,2526,Day of harvest listening to the sound of t...,0.417,joy,"[0, 0, 1, 0]",22
4,777,ðŸ˜³The intensity that play at is extraordina...,0.292,anger,"[1, 0, 0, 0]",14


#using glove word embadding

In [10]:
embeddings_dict = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.25d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      embeddings_dict[word] = vector

In [11]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))


#Preprocess Data

In [12]:
def tockanizer(tweet):
  tweet = df_train['Tweets'][ind].split()
  return tweet,len(tweet)

In [ ]:
tweetslist = []
for ind in df_train.index:
  tweet,length = tockanizer(df_train['Tweets'][ind])
  tweet.extend('x'*(maxLengthOfTweet - length))
  tweetslist.append(tweet)



print(tweetslist)


In [14]:
tweetseq= [];

for tweet in tweetslist:
  temp = []
  for word in tweet:
    if word != "x" and isinstance(word , str):
      try:
        temp.append(embeddings_dict[word.lower()])
      except KeyError:
        temp.append([0]*25)
    else:
      temp.append([0]*25)
    
  tweetseq.append(temp)

In [ ]:
tens=tf.Variable(tweetseq)
print(tens)

In [ ]:
labels = list(df_train['Encoding'])
labels=tf.convert_to_tensor(labels)
print(labels)

In [17]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(50,dropout=0.1,recurrent_dropout=0.1))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
model.build((None,maxLengthOfTweet,25))



In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                15200     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 204       
Total params: 15,404
Trainable params: 15,404
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(tweetseq,labels,
                  validation_split=0.2, epochs=100)